# Flask para gerar uma API

Tutorial de Flask: https://www.youtube.com/playlist?list=PLjSf4DcGBdiG52KoaYwfI1s5yRdtqeI-T

Instando as bibliotecas necessárias

!pip install flask-restful
!pip install flask
!pip install flask_jsonpify --user

In [ ]:
import pickle
import pandas as pd
import numpy as np
import time

from flask import Flask, request
from flask_restful import Api, Resource
from flask import Flask
from flask_jsonpify import jsonpify

Valores para prever o valor da casa

In [ ]:
# "100-323-4124.4324-0.4242-5".split("-")

In [ ]:
t = time.strftime('%x %X')
t

Função para escrever o arquivo de log de tudo o que está sendo feito

In [ ]:
def escreve_log(path='log.log', text=''):
    with open(path, 'a') as log:
        t = time.strftime('%x %X')
        log.write(t + str(text) + '\n')

Função para carregar o arquivo do modelo criado com o Pickle

In [ ]:
def open_pkl(path=''):
    escreve_log(text='comecou a carregar o pkl')
    file = open(path, 'rb')
    model = pickle.load(file)
    file.close
    escreve_log(text='terminou de carregar o pkl')
    return model

Classes de reposta para o acesso ao index (home) com tutorial e para acesso ao predict

In [ ]:
class Index(Resource):
    def get(self):
        escreve_log(text='acessou o index')
        return """Modelo Preditivo
        Este é o modelo para prever os valores de casa baseado nas seguintes caracteristicas
        Para conseguir sua previsão acesse a url/predict/<features>, onde as features devem ser substituidas pelos valores númericos de cada feature separados com "-"
        - LotArea
        - YearBuilt
        - OverallQual
        - WoodDeckSF
        - YrSold"""


class Prediction(Resource):
    def get(self, features):
        
        escreve_log(text='usuario iniciou uma prediction')
        features = str(features).split("-")
        X = np.array(features).reshape(1,5)
        escreve_log(text=f'valores:{X}' )
        df = pd.DataFrame(X, columns=['LotArea','YearBuilt','OverallQual','WoodDeckSF','YrSold'])
        model = open_pkl('model.pkl')
        
        yhat = model.predict(df)
        escreve_log(text=f'predicao:{yhat}')
        return jsonpify(yhat[0])

Função que inicia a Aplicação Flask

In [ ]:
def main():
    app = Flask(__name__)
    api = Api(app)
    
    api.add_resource(Index, '/')
    api.add_resource(Prediction, '/predict/<features>')

    app.run(port=1247, host='127.0.0.0')


main()